In [13]:
import pandas as pd
from datetime import datetime, timedelta

# Define column names
col_names = ['col1', 'col2', 'date', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9', 'col10', 'value', 'col12', 'col13', 'col14']

# Source of data NASA
# https://climate.nasa.gov/vital-signs/sea-level/

# Read the file into a DataFrame, skipping the first 50 lines
df = pd.read_csv('GMSL_TPJAOS_5.1_199209_202307.txt', delim_whitespace=True, skiprows=50, names=col_names)

# Function to parse fractional year
def parse_fractional_year(fractional_year):
    year = int(fractional_year)
    remainder = fractional_year - year
    base_date = datetime(year, 1, 1)
    result_date = base_date + timedelta(seconds=(base_date.replace(year=base_date.year + 1) - base_date).total_seconds() * remainder)
    return result_date

# Apply the function to the 'date' column
df['date'] = df['date'].apply(parse_fractional_year)

# Parse 'value' as a float
# Add value at 1992-01-01 to each value
# Divide each value by 1000 to get rise in meters
df['value'] = df['value'].astype(float).apply(lambda x: (x + 38.56) / 1000 )

# Keep only columns 'date' and 'value'
df = df[['date', 'value']]

print(df)

                           date    value
0    1993-01-05 04:58:03.936000 -0.00022
1    1993-01-15 02:56:30.912003 -0.00124
2    1993-01-25 00:54:57.887998 -0.00108
3    1993-02-03 22:53:56.399997 -0.00111
4    1993-02-13 20:52:23.375999 -0.00019
...                         ...      ...
1119 2023-06-02 17:31:32.304001  0.10347
1120 2023-06-12 15:29:59.279996  0.10409
1121 2023-06-22 13:28:26.255999  0.10457
1122 2023-07-02 11:26:53.232001  0.10491
1123 2023-07-12 09:25:20.207997  0.10514

[1124 rows x 2 columns]


In [14]:
# Convert DataFrame to list of lists
data_list = df.values.tolist()

# Convert datetime objects to strings
for i in range(len(data_list)):
    data_list[i][0] = data_list[i][0].strftime('%Y-%m-%d %H:%M:%S.%f')

print(data_list)

[['1993-01-05 04:58:03.936000', -0.00021999999999999887], ['1993-01-15 02:56:30.912003', -0.0012399999999999948], ['1993-01-25 00:54:57.887998', -0.0010799999999999983], ['1993-02-03 22:53:56.399997', -0.0011099999999999994], ['1993-02-13 20:52:23.375999', -0.00018999999999999773], ['1993-02-23 18:50:50.352002', 0.0008599999999999994], ['1993-03-05 16:49:17.327997', 0.0017300000000000039], ['1993-03-15 14:47:44.304000', 0.002270000000000003], ['1993-03-25 12:46:11.280002', 0.002490000000000002], ['1993-04-04 10:44:38.255997', 0.00243], ['1993-04-14 08:43:36.768003', 0.002190000000000005], ['1993-04-24 06:42:03.743999', 0.0018100000000000024], ['1993-05-04 04:40:30.720001', 0.0013700000000000045], ['1993-05-14 02:38:57.695997', 0.0011700000000000018], ['1993-05-24 00:37:24.671999', 0.0009200000000000017], ['1993-06-02 22:35:51.648002', 0.0013500000000000014], ['1993-06-12 20:34:50.160000', 0.001740000000000002], ['1993-06-22 18:33:17.136003', 0.0023200000000000004], ['1993-07-02 16:31:4

In [15]:
import json
import datetime

def write_to_file(filename, data):
    to_write = {
        'timestamp': datetime.datetime.now().isoformat(),
    }

    to_write.update(data)

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(to_write, f, indent=2, allow_nan=False)
        
write_to_file('../data/sealevels.json', { 'sealevels': data_list })